<a href="https://colab.research.google.com/github/anindabitm/ICLR-Workshop-Challenge-1-CGIAR-Computer-Vision-for-Crop-Disease/blob/master/ICLR_crop_diseases_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import re
import pickle
import os
import glob
import pandas as pd
from google.colab import drive
import cv2
from PIL import Image
from pathlib import Path
import fastai
from fastai.vision import *
from fastai.vision.models import *
import torch

In [2]:
fastai.__version__

'1.0.60'

In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir('/content/drive/My Drive/ICLR')

In [0]:
healthy_wheat=glob.glob('/content/drive/My Drive/ICLR/ICLR/train/train/healthy_wheat/*.*')
leaf_rust=glob.glob('/content/drive/My Drive/ICLR/ICLR/train/train/leaf_rust/*.*')
stem_rust=glob.glob('/content/drive/My Drive/ICLR/ICLR/train/train/stem_rust/*.*')

In [6]:
img_path=[]
img_path=img_path+healthy_wheat+leaf_rust+stem_rust
df=pd.DataFrame({'img_path':img_path})
df.shape

(1189, 1)

In [7]:
df.head()

,img_path
0,/content/drive/My Drive/ICLR/ICLR/train/train/...
1,/content/drive/My Drive/ICLR/ICLR/train/train/...
2,/content/drive/My Drive/ICLR/ICLR/train/train/...
3,/content/drive/My Drive/ICLR/ICLR/train/train/...
4,/content/drive/My Drive/ICLR/ICLR/train/train/...


In [8]:
df['img_path'] = df['img_path'].apply(lambda x: re.sub("/","",x,1))
df.head()

,img_path
0,content/drive/My Drive/ICLR/ICLR/train/train/h...
1,content/drive/My Drive/ICLR/ICLR/train/train/h...
2,content/drive/My Drive/ICLR/ICLR/train/train/h...
3,content/drive/My Drive/ICLR/ICLR/train/train/h...
4,content/drive/My Drive/ICLR/ICLR/train/train/h...


In [9]:
df['disease_class'] = df['img_path'].apply(lambda x: x.split("/")[7])
df['ext'] = df['img_path'].apply(lambda x: x.split(".")[-1])
df.head()

,img_path,disease_class,ext
0,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat,jpg
1,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat,jpg
2,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat,jpg
3,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat,jfif
4,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat,jpg


In [10]:
df=df[df.ext!='jfif']
df.drop('ext',axis=1,inplace=True)
df.head()

,img_path,disease_class
0,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat
1,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat
2,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat
4,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat
5,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat


In [11]:
df.shape

(876, 2)

In [0]:
df.reset_index(inplace=True)

In [0]:
df.drop('index',axis=1,inplace=True)

In [14]:
df.head()

,img_path,disease_class
0,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat
1,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat
2,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat
3,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat
4,content/drive/My Drive/ICLR/ICLR/train/train/h...,healthy_wheat


In [15]:
test_img_path=glob.glob('/content/drive/My Drive/ICLR/ICLR/test/test/*.*')
test_df=pd.DataFrame({'test_img_path':test_img_path})
test_df['ext'] = test_df['test_img_path'].apply(lambda x: x.split(".")[-1])
test_df=test_df[test_df.ext!='jfif']
test_df.shape

(610, 2)

In [16]:
test_df['test_img_path'] = test_df['test_img_path'].apply(lambda x: re.sub("/","",x,1))
test_df.drop('ext',axis=1,inplace=True)
test_df.reset_index(inplace=True)
test_df.drop('index',axis=1,inplace=True)
test_df.head()

,test_img_path
0,content/drive/My Drive/ICLR/ICLR/test/test/008...
1,content/drive/My Drive/ICLR/ICLR/test/test/085...
2,content/drive/My Drive/ICLR/ICLR/test/test/00A...
3,content/drive/My Drive/ICLR/ICLR/test/test/01O...
4,content/drive/My Drive/ICLR/ICLR/test/test/0K9...


In [17]:
df['disease_class'].value_counts()

stem_rust        376
leaf_rust        358
healthy_wheat    142
Name: disease_class, dtype: int64

In [0]:
def create_databunch(valid_idx):
  test = ImageList.from_df(test_df,path='/')
  data = (ImageList.from_df(df, path='/')
              .split_by_idx(valid_idx)
              .label_from_df(cols='disease_class')
              .add_test(test)
              .transform(get_transforms(flip_vert=False, max_rotate=20.0), size=256)
              .databunch(path='.', bs=32)
              .normalize(imagenet_stats)
            )
  return data

In [0]:
model_densenet=models.densenet201

In [0]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3,shuffle=True,random_state=123)
acc_val=[]
i = 0
for train_idx, valid_idx in kf.split(df):
      print("Validation set:--->  ",i+1)
      data = create_databunch(valid_idx)
      learn = cnn_learner(data, model_densenet, metrics=[accuracy])
      learn.fit_one_cycle(1, 1e-3)
      learn.unfreeze()
      learn.fit_one_cycle(9, slice(1e-4,1e-2))
      loss,acc=learn.validate()
      acc_val.append(acc.numpy())
      i=i+1

print("The validation accuracy is: ",np.mean(acc_val))

Validation set:--->   1


Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/checkpoints/densenet201-c1103571.pth


epoch,train_loss,valid_loss,accuracy,time
0,1.254156,0.700012,0.756849,05:09


epoch,train_loss,valid_loss,accuracy,time
0,1.052977,0.630963,0.797945,02:21
1,0.949462,1.505078,0.736301,02:15
2,0.824370,0.588844,0.839041,02:15
3,0.696334,0.541852,0.825342,02:16
4,0.550471,0.350742,0.890411,02:13
5,0.428061,0.426682,0.900685,02:14
6,0.352092,0.465825,0.900685,02:12
7,0.290502,0.462767,0.883562,02:14
8,0.223662,0.432226,0.883562,02:13


Validation set:--->   2


epoch,train_loss,valid_loss,accuracy,time
0,1.488695,0.704219,0.760274,02:12


epoch,train_loss,valid_loss,accuracy,time
0,0.979333,0.793467,0.777397,02:12
1,0.926235,1.401847,0.756849,02:13
2,0.808649,0.556457,0.845890,02:14
3,0.691980,0.527485,0.828767,02:12
4,0.604026,0.532222,0.835616,02:15
5,0.486363,0.487057,0.863014,02:11
6,0.393344,0.387635,0.914384,02:13
7,0.304617,0.423120,0.890411,02:12
8,0.241812,0.422804,0.893836,02:15


Validation set:--->   3


epoch,train_loss,valid_loss,accuracy,time
0,1.487599,0.678378,0.726027,02:11


epoch,train_loss,valid_loss,accuracy,time
0,0.920834,0.964422,0.722603,02:15
1,0.777105,1.198053,0.811644,02:13


In [0]:
submission = pd.DataFrame({'ID': [s.split('.')[0] for s in os.listdir('/content/drive/My Drive/ICLR/ICLR/test/test')  if s.split('.')[1] != 'jfif']})
for i, c in enumerate(learn.data.classes):
  submission[c] = preds_test[:,i]
submission = submission[['ID', 'leaf_rust', 'stem_rust', 'healthy_wheat']] # Get same order as sample sub
submission.head()

,ID,leaf_rust,stem_rust,healthy_wheat
0,008FWT,1.0,0.0,0.0
1,085IEC,0.0,0.0,1.0
2,00AQXY,0.0,1.0,0.0
3,01OJZX,1.0,0.0,0.0
4,0K9T9T,0.0,1.0,0.0


In [0]:
submission.to_csv('/content/drive/My Drive/ICLR/submission.csv', index = False)